In [8]:
import torch
import torchaudio
from IPython.display import Audio, display

### Loading Model

In [ ]:
facodec = torch.hub.load(repo_or_dir='ex3ndr/supervoice-facodec', model='facodec', trust_repo = True)

# Loading file
Provide custom file or try the sample

In [4]:
from ipywidgets import FileUpload
upload = FileUpload(multiple=False)
upload

FileUpload(value=(), description='Upload')

In [23]:
def load_mono_audio(path):
    # Load audio
    audio, sr = torchaudio.load(path)

    # Resample
    if sr != 16000:
        audio = torchaudio.transforms.Resample(sr, 16000)(audio)
        sr = 16000

    # Convert to mono
    if audio.shape[0] > 1:
        audio = audio.mean(dim=0, keepdim=True)

    # Convert to single dimension
    audio = audio[0]

    return audio

# Load
if len(upload.value) == 1:
    with open("eval_vocoder.out", "w+b") as i:
        i.write(upload.value[0].content)
    source = load_mono_audio("eval_vocoder.out")
else:
    source = load_mono_audio("./eval/eval_0.wav")
display(Audio(data=source, rate=16000))

## Encoding file, getting codes and speaker embeddings

In [24]:
# Get codes for an audio
prosody_code, cotent_code, residual_code, spk_embs = facodec.encode(source)

print("Speaker Embedding: ", spk_embs.shape)
print("Prosody Code: ", prosody_code.shape)
print("Content Code: ", cotent_code.shape)
print("Residual Code: ", residual_code.shape)

Speaker Embedding:  torch.Size([256])
Prosody Code:  torch.Size([1, 589])
Content Code:  torch.Size([2, 589])
Residual Code:  torch.Size([3, 589])


## Decoding back

In [25]:
reconstructed = facodec.decode(prosody_code, cotent_code, residual_code, spk_embs)
display(Audio(data=reconstructed, rate=16000))

## Speaker Changing
How to change speaker style

In [33]:
# Load Speaker
speaker_style = load_mono_audio("./eval/eval_2.mp3")
speaker_embedding = facodec.speaker_embedding(speaker_style)
display(Audio(data=speaker_style, rate=16000))

In [34]:
reconstructed_styled = facodec.speech_convert(prosody_code, cotent_code, speaker_embedding)
display(Audio(data=reconstructed_styled, rate=16000))